In [5]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [6]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [7]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCNO', 'des_col': 'scno','src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUPP_RELEASE_DATE', 'des_col': 'supp_release_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'timestamp'},
    {'src_col': 'PHONE', 'des_col': 'phone', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EMAIL', 'des_col': 'email', 'src_dtype': 'String', 'des_dtype': 'DECIMAL'},
    {'src_col': 'SERVICE_STATUS', 'des_col': 'service_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NAME', 'des_col': 'name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_1', 'des_col': 'address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_2', 'des_col': 'address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_3', 'des_col': 'address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDRESS_4', 'des_col': 'address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CATEGORY', 'des_col': 'category', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LOAD_UNIT', 'des_col': 'load_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE', 'des_col': 'phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'GRP', 'des_col': 'grp', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OFFICE_ID', 'des_col': 'office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TERMCOVER_NUM', 'des_col': 'termcover_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TERMCOVER_IMP', 'des_col': 'termcover_imp', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FEEDER_NUM', 'des_col': 'feeder_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANS_SER_NUM', 'des_col': 'trans_ser_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANS_STRUC_CODE', 'des_col': 'trans_struc_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TRANS_LOCATION', 'des_col': 'trans_location', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'POL_NUM', 'des_col': 'pol_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBSTATION_NUM', 'des_col': 'substation_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_1', 'des_col': 'alternate_address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_2', 'des_col': 'alternate_address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_3', 'des_col': 'alternate_address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ALTERNATE_ADDRESS_4', 'des_col': 'alternate_address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILLING_STATUS', 'des_col': 'billing_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CONTRACTED_LOAD', 'des_col': 'contracted_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CONNECTED_LOAD', 'des_col': 'connected_load', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'LOAD_TYPE', 'des_col': 'load_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CAPACITOR_AVAIL', 'des_col': 'capacitor_avail', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BILL_MODE', 'des_col': 'bill_mode', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ED_FLAG', 'des_col': 'ed_flag', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ERO_CODE', 'des_col': 'ero_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SPECIFIED_KV', 'des_col': 'specified_kv', 'src_dtype': 'DECIMAL', 'des_dtype': 'integer'},
    {'src_col': 'ACTUAL_KV', 'des_col': 'actual_kv', 'src_dtype': 'DECIMAL', 'des_dtype': 'integer'},
    {'src_col': 'LFLOAD', 'des_col': 'lfload', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'DEMAND_EXEMPTION', 'des_col': 'demand_exemption', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SERVICE_TYPE', 'des_col': 'service_type', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECFLOAT', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LEADER_ID', 'des_col': 'leader_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOCIAL_STATUS_ID', 'des_col': 'social_status_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'EFFECT_DATE', 'des_col': 'effect_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'timestamp'},
    {'src_col': 'LETTER_NO', 'des_col': 'letter_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AREA_ID', 'des_col': 'area_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SURNAME', 'des_col': 'surname', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LETTER_DATE', 'des_col': 'letter_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'timestamp'},
    {'src_col': 'MTRCURRENTSTAT', 'des_col': 'mtrcurrentstat', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBGROUP', 'des_col': 'subgroup', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RET_MONTH', 'des_col': 'ret_month', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REMARKS', 'des_col': 'remarks', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPROVED_BY', 'des_col': 'approved_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ADDL_SDAMT', 'des_col': 'addl_sdamt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'PCB_NUMBER', 'des_col': 'pcb_number', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PCB_DATE', 'des_col': 'pcb_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'timestamp'},
    {'src_col': 'BCRC_NO', 'des_col': 'bcrc_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'BCRC_DATE', 'des_col': 'bcrc_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'timestamp'},
    {'src_col': 'BANK_ID', 'des_col': 'bank_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CSCNO', 'des_col': 'cscno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_POL_NUM', 'des_col': 'new_pol_num', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_STRUC_CODE', 'des_col': 'new_struc_code', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_AREA_ID', 'des_col': 'new_area_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_AREACODE', 'des_col': 'new_areacode', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_NAME', 'des_col': 'new_name', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_SURNAME', 'des_col': 'new_surname', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_FHPNAME', 'des_col': 'new_fhpname', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ADDRESS_1', 'des_col': 'new_address_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ADDRESS_2', 'des_col': 'new_address_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ADDRESS_3', 'des_col': 'new_address_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ADDRESS_4', 'des_col': 'new_address_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_GRP', 'des_col': 'new_grp', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_SUBGROUP', 'des_col': 'new_subgroup', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_CSCNO', 'des_col': 'new_cscno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TO_BE_APPROVEDBY', 'des_col': 'to_be_approvedby', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'REJECT_REASON', 'des_col': 'reject_reason', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_COVER_SEALNO_1', 'des_col': 'old_cover_sealno_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_COVER_SEALNO_2', 'des_col': 'old_cover_sealno_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_COVER_SEALNO_3', 'des_col': 'old_cover_sealno_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_COVER_SEALNO_4', 'des_col': 'old_cover_sealno_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_TERMINAL_SEALNO_1', 'des_col': 'old_terminal_sealno_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_TERMINAL_SEALNO_2', 'des_col': 'old_terminal_sealno_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_COVER_SEALNO_1', 'des_col': 'new_cover_sealno_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_COVER_SEALNO_2', 'des_col': 'new_cover_sealno_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_COVER_SEALNO_3', 'des_col': 'new_cover_sealno_3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_COVER_SEALNO_4', 'des_col': 'new_cover_sealno_4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_TERMINAL_SEALNO_1', 'des_col': 'new_terminal_sealno_1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_TERMINAL_SEALNO_2', 'des_col': 'new_terminal_sealno_2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_METERNO', 'des_col': 'new_meterno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_METERNO', 'des_col': 'old_meterno', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'APPROVALS', 'des_col': 'approvals', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLD_ACCOUNT_NO', 'des_col': 'old_account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ACCOUNT_NO', 'des_col': 'new_account_no', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'OLDMETERDIGIT', 'des_col': 'oldmeterdigit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEWMETERDIGIT', 'des_col': 'newmeterdigit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CYCLE', 'des_col': 'cycle', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'DHARI_YEAR', 'des_col': 'dhari_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RDG_YEAR', 'des_col': 'rdg_year', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ZONE_ID', 'des_col': 'zone_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'NEW_ZONE_ID', 'des_col': 'new_zone_id', 'src_dtype': 'String', 'des_dtype': 'String'}
    
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.CONSUMER_MISC_HIS', 'public', 'consumer_misc_his', cols_map)
print("Total Migrated Records (consumer_misc_his): " + str(migrated_row_count))


Total Migrated Records (consumer_misc_his): Total Src Records: 89575 Total migrated Records to dest: 89575
